# Draw Interst Area Boundaries On Videos

In [1]:
import cv2
import os
import glob
import numpy as np
import time

In [2]:
raw_video_dir = '../ROM_raw_videos'
raw_video_sub_dir = os.listdir(raw_video_dir)
raw_video_sub_dir.sort()
raw_video_sub_dir

['Oct_02',
 'Oct_03',
 'Oct_05',
 'Sep_12',
 'Sep_13',
 'Sep_14',
 'Sep_17',
 'Sep_18',
 'Sep_19',
 'Sep_20',
 'Sep_21',
 'Sep_24',
 'Sep_25',
 'Sep_26',
 'Sep_27',
 'Sep_28']

In [3]:
output_video_dir = '../ROM_raw_videos_with_interst_area_test'

In [ ]:

raw_video_sub_dir = ['Sep_17']
for sub_dir in raw_video_sub_dir:
    files_path = glob.glob(os.path.join(raw_video_dir, sub_dir, '*.mp4'))
    print('{} files in {}'.format(len(files_path), sub_dir))
    
    for f_path in files_path:
        if '1300' in f_path and 'Camera1' in f_path:
            f_name = f_path.split('/')[-1]
            print(''.format(f_name))

            # Get Camera index
            camera_index = int(f_name[6])
            print('Camera index: {}'.format(camera_index))

            # Get frame size
            camera = cv2.VideoCapture(f_path)
            (grabbed, frame) = camera.read()
            original_h, original_w, channels= frame.shape

            # Define the polygon of Whole Interest Area for videos from Camera1 or Camera2
            if camera_index == 1:
                # crop frame: Camera1
                top_edge = int(original_h*(1/10))
                down_edge = int(original_h*1)
                left_edge = int(original_w*(1/5))
                right_edge = int(original_w*(4/5))
                point_1 = [left_edge, top_edge]
                point_2 = [left_edge, down_edge]
                point_3 = [right_edge, down_edge]
                point_4 = [right_edge, top_edge]
                whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
            elif camera_index == 2:
                # crop frame: Camera2
                top_edge = int(original_h*(1/10))
                down_edge = int(original_h*(4/5))
                left_edge = int(original_w*(2.5/5))
                right_edge = int(original_w*(1))
                point_1 = [left_edge, top_edge]
                point_2 = [left_edge, down_edge]
                point_3 = [right_edge, down_edge]
                point_4 = [right_edge, top_edge]
                whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
            else:
                # crop frame: test video
                top_edge = int(original_h*(1/10))
                down_edge = int(original_h*1)
                left_edge = int(original_w*(1/5))
                right_edge = int(original_w*(4/5))
                print('Polygon: Video not from Camera1 or Camera2!')
                point_1 = [left_edge, top_edge]
                point_2 = [left_edge, down_edge]
                point_3 = [right_edge, down_edge]
                point_4 = [right_edge, top_edge]
                whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])

            # Define the polygon of Core Interest Area for videos from Camera1 or Camera2
            cropped_w = right_edge - left_edge
            cropped_h = down_edge - top_edge
            if camera_index == 1:
                # polygon for Camera1
                point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
                point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
                point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
                point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
                point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
                core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])
            elif camera_index == 2:
                # polygon for Camera2
                point_1 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.05 * cropped_h)]
                point_2 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.65 * cropped_h)]
                point_3 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.75 * cropped_h)]
                point_4 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.05 * cropped_h)]
                core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
            else:
                # polygon for test video
                point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
                point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
                point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
                point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
                point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
                print('Polygon: Video not from Camera1 or Camera2!')
                core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])

            # get output video file name
            # (XVID is more preferable. MJPG results in high size video. X264 gives very small size video)
            # https://docs.opencv.org/3.1.0/dd/d43/tutorial_py_video_display.html
            #fourcc = cv2.VideoWriter_fourcc(*'X264')

            output_video_sub_dir = os.path.join(output_video_dir, sub_dir)
            if not os.path.exists(output_video_sub_dir):
                os.makedirs(output_video_sub_dir)

            output_video_filename = os.path.join(output_video_sub_dir,'{}_draw_interest_area.avi'.format(f_name.split('.mp4')[0]))
            out_camera = cv2.VideoWriter(output_video_filename, int(camera.get(cv2.CAP_PROP_FOURCC)), camera.get(cv2.CAP_PROP_FPS), (original_w, original_h))


            # loop over the frames of the video
            camera = cv2.VideoCapture(f_path)
            total_frame_number = camera.get(cv2.CAP_PROP_FRAME_COUNT)
            print('Total frame number: {}'.format(total_frame_number))
            start_time = time.time()

            for frame_count in range(int(total_frame_number)):
                # get time
                frame_time = camera.get(cv2.CAP_PROP_POS_MSEC)

                if frame_count % 2000 == 0:
                    print('Processing frame: {}'.format(frame_count))
                    print('Elapsed time: {}s'.format(time.time() - start_time))
                (grabbed, frame) = camera.read()
                if grabbed == True:

                    cv2.drawContours(frame, [whole_interest_area_polygon], -1, (255, 255, 0), 6, cv2.LINE_AA)
                    cv2.drawContours(frame, [core_interest_area_polygon], -1, (255, 0, 0), 6, cv2.LINE_AA)

                    s_temp = int(frame_time/1000)
                    m = s_temp // 60
                    s = s_temp % 60
                    cv2.putText(frame, "Original video time: {}m {}s".format(m, s), (10, 1060), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

                    if camera_index == 1:
                        cv2.putText(frame, "Camera View", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                        cv2.putText(frame, "Whole Interest Area", (left_edge+10, top_edge+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
                        cv2.putText(frame, "Core Interest", (point_1[0]+10, point_1[1]+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
                        cv2.putText(frame, "Area", (point_1[0]+10, point_1[1]+130), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
                    elif camera_index == 2:
                        cv2.putText(frame, "Camera View", (800, 1000), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                        cv2.putText(frame, "Whole Interest Area", (left_edge+10, down_edge-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
                        cv2.putText(frame, "Core Interest Area", (point_2[0]+10, point_2[1]-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
                    else:
                        pass
                    # save processed videos
                    out_camera.write(frame)
                else:
                    # Pass this frame if cannot grab an image.
                    print('Frame: {}, grabbed={}'.format(frame_count, grabbed))
        

6 files in Sep_17

Camera index: 1
Total frame number: 91884.0
Processing frame: 0
Elapsed time: 4.982948303222656e-05s
Processing frame: 2000
Elapsed time: 97.65947389602661s
Processing frame: 4000
Elapsed time: 192.7150013446808s
Processing frame: 6000
Elapsed time: 293.24481177330017s
Processing frame: 8000
Elapsed time: 393.25816345214844s
Processing frame: 10000
Elapsed time: 488.88309693336487s
Processing frame: 12000
Elapsed time: 585.4023911952972s
Processing frame: 14000
Elapsed time: 679.3073678016663s
Processing frame: 16000
Elapsed time: 773.7013690471649s
Processing frame: 18000
Elapsed time: 869.046807050705s
Processing frame: 20000
Elapsed time: 970.0736854076385s
Processing frame: 22000
Elapsed time: 1066.6092867851257s
Processing frame: 24000
Elapsed time: 1169.443959236145s
Processing frame: 26000
Elapsed time: 1264.332409620285s
Processing frame: 28000
Elapsed time: 1359.219486951828s
Processing frame: 30000
Elapsed time: 1460.7626564502716s
Processing frame: 32000
E

In [ ]:
import cv2
import os
import glob
import numpy as np
import time
import argparse

if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("-v", "--video", default='', help="path to the video file")
    ap.add_argument("-o", "--output_directory", default='', help="directory to save processed video")
    args = vars(ap.parse_args())

    if args.get("video", None) is None:
        raise Error("No input video!!")
    # otherwise, we are reading from a video file
    else:
        f_path = args["video"]

    f_name = f_path.split('/')[-1]
    print(''.format(f_name))
    output_video_dir = args['output_directory']#'../ROM_raw_videos_with_interst_area_test'
    sub_dir = f_path.split('/')[-2]
    
    # Get Camera index
    camera_index = int(f_name[6])
    print('Camera index: {}'.format(camera_index))

    # Get frame size
    camera = cv2.VideoCapture(f_path)
    (grabbed, frame) = camera.read()
    original_h, original_w, channels= frame.shape

    # Define the polygon of Whole Interest Area for videos from Camera1 or Camera2
    if camera_index == 1:
        # crop frame: Camera1
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*1)
        left_edge = int(original_w*(1/5))
        right_edge = int(original_w*(4/5))
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    elif camera_index == 2:
        # crop frame: Camera2
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*(4/5))
        left_edge = int(original_w*(2.5/5))
        right_edge = int(original_w*(1))
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    else:
        # crop frame: test video
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*1)
        left_edge = int(original_w*(1/5))
        right_edge = int(original_w*(4/5))
        print('Polygon: Video not from Camera1 or Camera2!')
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])

    # Define the polygon of Core Interest Area for videos from Camera1 or Camera2
    cropped_w = right_edge - left_edge
    cropped_h = down_edge - top_edge
    if camera_index == 1:
        # polygon for Camera1
        point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
        point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
        point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
        point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
        point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])
    elif camera_index == 2:
        # polygon for Camera2
        point_1 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.05 * cropped_h)]
        point_2 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.65 * cropped_h)]
        point_3 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.75 * cropped_h)]
        point_4 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.05 * cropped_h)]
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    else:
        # polygon for test video
        point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
        point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
        point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
        point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
        point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
        print('Polygon: Video not from Camera1 or Camera2!')
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])

    # get output video file name
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    output_video_sub_dir = args['output_directory']#os.path.join(output_video_dir, sub_dir)
    if not os.path.exists(output_video_sub_dir):
        os.makedirs(output_video_sub_dir)

    output_video_filename = os.path.join(output_video_sub_dir,'{}_draw_interest_area.avi'.format(f_name.split('.mp4')[0]))
    out_camera = cv2.VideoWriter(output_video_filename, fourcc, camera.get(cv2.CAP_PROP_FPS), (original_w, original_h))

    # loop over the frames of the video
    camera = cv2.VideoCapture(f_path)
    total_frame_number = camera.get(cv2.CAP_PROP_FRAME_COUNT)
    print('Total frame number: {}'.format(total_frame_number))
    start_time = time.time()

    for frame_count in range(int(total_frame_number)):
        if frame_count % 2000 == 0:
            print('Processing frame: {}'.format(frame_count))
            print('Elapsed time: {}s'.format(time.time() - start_time))
        (grabbed, frame) = camera.read()
        if grabbed == True:

            cv2.drawContours(frame, [whole_interest_area_polygon], -1, (255, 255, 0), 6, cv2.LINE_AA)
            cv2.drawContours(frame, [core_interest_area_polygon], -1, (255, 0, 0), 6, cv2.LINE_AA)
            if camera_index == 1:
                cv2.putText(frame, "Camera View", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                cv2.putText(frame, "Whole Interest Area", (left_edge+10, top_edge+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
                cv2.putText(frame, "Core Interest", (point_1[0]+10, point_1[1]+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
                cv2.putText(frame, "Area", (point_1[0]+10, point_1[1]+130), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
            elif camera_index == 2:
                cv2.putText(frame, "Camera View", (800, 1000), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                cv2.putText(frame, "Whole Interest Area", (left_edge+10, down_edge-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)
                cv2.putText(frame, "Core Interest Area", (point_2[0]+10, point_2[1]-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 4)
            else:
                pass
            # save processed videos
            out_camera.write(frame)
        else:
            # Pass this frame if cannot grab an image.
            print('Frame: {}, grabbed={}'.format(frame_count, grabbed))